In [324]:
import os
import pandas as pd

In [325]:
df = pd.read_csv('Data/spotify_millsongdata.csv')

In [326]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [327]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [328]:
df.shape

(57650, 4)

In [329]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [330]:
df = df.sample(6500).drop('link', axis=1, errors='ignore').reset_index(drop=True)


In [331]:
df.head()

,artist,song,text
0,Pogues,Modern World,Mary's sellin' flowers on a stall in EC1 \r\n...
1,Ufo,Rock Bottom,"Seventeen a nature's queen, know what I mean ..."
2,Isley Brothers,Stop! In The Name Of Love,Stop! In The Name Of Love \r\nBefore you brea...
3,Quietdrive,I Lie Awake,I lie awake \r\nBecause I'm scared \r\nI'm n...
4,Neil Diamond,I'm Guilty,Whenever I get down \r\nI look to someone els...


In [332]:
df["text"][0]

"Mary's sellin' flowers on a stall in EC1  \r\nTonight she'll be out lookin' for some fun  \r\nHer mate, Gerry's in the basement, he hasn't got a home  \r\nBut prides himself that he's got most things done  \r\nMeanwhile, I'm up on the roof  \r\nWaitin' for the sun to shine  \r\nI haven't got an excuse  \r\nSo, I'll just keep my head down  \r\nDavid's in the city, he's just made a deal  \r\nAnd the boys they want to take his trousers down  \r\nSo they buy some little pills which will make poor David ill  \r\nAnd they find themselves in some club downtown  \r\nMeanwhile, I got off the roof  \r\n'cause the rain keeps pouring down  \r\nAnd I haven't got a clue  \r\nWhy there's faith in all these clowns  \r\nThis is the modern world  \r\nThis is the modern world  \r\nJim and Jane hit the grapevine 'cause they're lookin' for a party  \r\nA secret number knows where it can be found  \r\nSo they pick up Pete and Sheila, then they head down the M40  \r\nBut someone said they closed the country

In [333]:
df['text']=df['text'] = df['text'].str.lower().str.replace(r'^\w\s', ' ').replace(r'\n', '', regex=True)

In [334]:
df.head()

,artist,song,text
0,Pogues,Modern World,mary's sellin' flowers on a stall in ec1 \rto...
1,Ufo,Rock Bottom,"seventeen a nature's queen, know what i mean ..."
2,Isley Brothers,Stop! In The Name Of Love,stop! in the name of love \rbefore you break ...
3,Quietdrive,I Lie Awake,i lie awake \rbecause i'm scared \ri'm not a...
4,Neil Diamond,I'm Guilty,whenever i get down \ri look to someone else ...


In [335]:
import nltk
from nltk.stem.porter import PorterStemmer

In [336]:
stemmer = PorterStemmer()

In [337]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(i) for i in token]
    return " ".join(a)

In [338]:
token("you are running faster than the others")

'you are run faster than the other'

In [339]:
df['text'] = df['text'].apply(token)

In [340]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [341]:
tfid = TfidfVectorizer( analyzer='word', stop_words='english') 

In [342]:
matrix = tfid.fit_transform(df['text'])

In [343]:
similar = cosine_similarity(matrix)

In [344]:
similar[0]

array([1.        , 0.01121946, 0.03509332, ..., 0.07080833, 0.00559091,
       0.00614794])

In [345]:
print(df['song'].sample(10).values)


['If I Gave My Heart To You' 'The Speed Of Pain' 'Paradise'
 'I Could Not Love You More' 'Hear My Train A Coming' 'You Alone'
 'Foreigner Suite' 'Beyond The Wheel' 'Rocketman' 'Queen Of Torture']


In [349]:
def recommender(song_name, similar_matrix, df):
    matches = df[df['song'].str.lower() == song_name.lower()]
    if matches.empty:
        return f"❌ The song '{song_name}' was not found in the dataset."
    
    idx = matches.index[0]
    distances = sorted(list(enumerate(similar_matrix[idx])), key=lambda x: x[1], reverse=True)
    
    songs = []
    for i in distances[1:6]:  # skip itself
        songs.append(df.iloc[i[0]]['song'])
    return songs


In [351]:
recommender("If I Gave My Heart To You", similar, df)


['Live Forever',
 "Don't Go Breaking My Heart",
 'Restless',
 'Hearts',
 "Don't Let It Go To Your Heart"]